In [2]:
from spectrumdevice import SpectrumAWGCard

# digitiser_1 = SpectrumDigitiserCard(device_number=0)
awg_1 = SpectrumAWGCard(device_number=0)

ValueError: 484913 is not a valid ModelNumber

In [ ]:
from spectrumdevice import MockSpectrumDigitiserCard, MockSpectrumDigitiserStarHub, MockSpectrumAWGCard
from spectrumdevice.settings import ModelNumber

mock_digitiser = MockSpectrumDigitiserCard(
    device_number=0,
    model=ModelNumber.TYP_M2P5966_X4,
    mock_source_frame_rate_hz=10.0,
    num_modules=2,
    num_channels_per_module=4
)
mock_hub = MockSpectrumDigitiserStarHub(device_number=0, child_cards=[mock_digitiser], master_card_index=0)
mock_awg = MockSpectrumAWGCard(
    device_number=0,
    model=ModelNumber.TYP_M2P6560_X4,
    num_modules=1,
    num_channels_per_module=1
)

In [ ]:
from time import sleep

from numpy import int16

from spectrumdevice.devices.awg.awg_card import SpectrumAWGCard
from spectrumdevice.devices.awg.synthesis import make_full_scale_sine_waveform
from spectrumdevice.devices.mocks import MockSpectrumAWGCard
from spectrumdevice.settings import (
    TriggerSettings,
    TriggerSource,
    GenerationSettings,
    OutputChannelFilter,
    ModelNumber,
)
from spectrumdevice.settings.channel import OutputChannelStopLevelMode
from spectrumdevice.settings.device_modes import GenerationMode

PULSE_RATE_HZ = 200
NUM_TRANSMISSIONS = 5
NUM_CYCLES_PER_TRANSMISSION = 3
FREQUENCY_HZ = 1e3
AMPLITUDE_V = 1.0
SAMPLE_RATE = 125000000


def awg_single_restart_mode_example(mock_mode: bool) -> None:

    # create a connection to a mock or real AWG card
    if not mock_mode:
        card = SpectrumAWGCard(device_number=0)
    else:
        card = MockSpectrumAWGCard(
            device_number=0, model=ModelNumber.TYP_M2P6560_X4, num_modules=1, num_channels_per_module=1
        )

    sample_rate_in_hz = SAMPLE_RATE
    number_of_generations = NUM_TRANSMISSIONS

    # create a waveform to generate
    t, analog_wfm = make_full_scale_sine_waveform(
        frequency_in_hz=FREQUENCY_HZ,
        sample_rate_in_hz=sample_rate_in_hz,
        num_cycles=NUM_CYCLES_PER_TRANSMISSION,
        dtype=int16,
    )

    # configure signal generation
    generation_settings = GenerationSettings(
        generation_mode=GenerationMode.SPC_REP_STD_SINGLERESTART,
        waveform=analog_wfm,
        sample_rate_in_hz=sample_rate_in_hz,
        num_loops=number_of_generations,
        enabled_channels=[0],
        signal_amplitudes_in_mv=[int(round((AMPLITUDE_V * 1000)))],
        dc_offsets_in_mv=[0],
        output_filters=[OutputChannelFilter.LOW_PASS_70_MHZ],
        stop_level_modes=[OutputChannelStopLevelMode.SPCM_STOPLVL_ZERO],
    )
    card.configure_generation(generation_settings)

    # configure triggering (here we configure an external trigger, but actually we will be forcing
    # the trigger event to occur from software on line 72)
    trigger_settings = TriggerSettings(
        trigger_sources=[TriggerSource.SPC_TMASK_EXT0],
    )
    card.configure_trigger(trigger_settings)

    # start the card and then force a trigger for each generation we want to perform
    # we are using GenerationMode.SPC_REP_STD_SINGLERESTART so the whole waveform will be generated each time the card
    # is trigger, until "num_loops" triggers have been detected.
    card.start()

    # force triggers at the requested rate. Remove if real external trigger present.
    for _ in range(number_of_generations):
        card.force_trigger()
        sleep(1 / PULSE_RATE_HZ)
        print("generated pulse")
    card.stop()
    card.disconnect()


if __name__ == "__main__":
    # change mock_mode to False to connect to a real card
    awg_single_restart_mode_example(mock_mode=False)

Python Version: 3.11.8 on Windows



SpectrumApiCallFailed: "spcm_dwGetParam_i32" failed with "The used handle is not valid." {'Spectrum API error code: 0x00000009'}

In [ ]:
card = SpectrumAWGCard(device_number=0)
card.stop()
card.disconnect()

ValueError: 484913 is not a valid ModelNumber

## *New Markus Program*

In [1]:
# Author: Marcus Culemann
# Version: March 2024

import numpy as np
from spectrum_card_interface import SpectrumCardInterface
from spectrum_card_waveform_generation_tool import SpectrumCardWaveformTool

######################################################################################################
## This file automatically sets up the Spectrum Card according to the chosen settings.              ##
######################################################################################################

#####################
# Waveform settings #
#####################

freq_array_horizontal = np.array([200e6])
freq_array_vertical = np.array([60e6, 61e6])

amplitude_array_horizontal = np.ones_like(freq_array_horizontal)
amplitude_array_vertical = np.ones_like(freq_array_vertical)

rng = np.random.default_rng(seed=13438354524)
phase_array_horizontal = rng.uniform(size=freq_array_horizontal.size)*2*np.pi
phase_array_vertical = rng.uniform(size=freq_array_vertical.size)*2*np.pi

#################
# Card Settings #
#################
spectrum_card = SpectrumCardInterface()
spectrum_card.ch1_enabled = True
spectrum_card.ch2_enabled = False
spectrum_card.amplitude_ch1 = 2500
spectrum_card.amplitude_ch2 = 158
spectrum_card.num_loops = 1000
spectrum_card.memory_size = 1024**2 * 20
spectrum_card.sampleRate = 625

########################################
# Generate the waveform and setup card #
########################################

waveform_generator = SpectrumCardWaveformTool(freq_array_horizontal=freq_array_horizontal,
                                     freq_array_vertical=freq_array_vertical,
                                     amplitude_array_horizontal=amplitude_array_horizontal,
                                     amplitude_array_vertical=amplitude_array_vertical,
                                     phase_array_horizontal=phase_array_horizontal,
                                     phase_array_vertical=phase_array_vertical,
                                     num_samples=spectrum_card.memory_size,
                                     sample_rate=spectrum_card.sampleRate
                                     )

spectrum_card.replay_data_ch1 = waveform_generator.getWaveformHorizontal()
spectrum_card.replay_data_ch2 = waveform_generator.getWaveformVertical()

spectrum_card.setupCard()
spectrum_card.startReplay()

spectrum_card.closeCard()

ModuleNotFoundError: No module named 'spectrum_card_interface'